In [2]:
!pip install pytorch_pretrained_bert
!pip install --upgrade transformers 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 73.9 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.106
    Uninstalling botocore-1.34.106:
      Successfully uninstalled botocore-1.34.106
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.0 requires aiohttp<4.0.0,>=3.9.2, but you have aiohttp 3.9.1 which is incompatible.
aiobotocore 2.13.0 requires botocore<1.34.107,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 55.2 MB/s eta 0:00:0000:0100:01
  Attempting uninsta

In [3]:
#for augmentation 
!pip install nlpaug
import nlpaug.augmenter.word as naw

import nltk
nltk.download('wordnet')
nltk.download('omw')
nltk.data.path.append("/kaggle/working/nltk_data")


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /usr/share/nltk_data...
[nltk_data]   Package omw is already up-to-date!


In [4]:
!pip install tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tools: filename=tools-0.1.9-py3-none-any.whl size=46731 sha256=782871292801c571f872ea0bbbe39ff5dac73a4f71ffd450c0dc309491f54576
  Stored in directory: /root/.cache/pip/wheels/c0/d0/70/a33bd4bed2af4f7038b038c16faab552cd0e9d9f4125223a71
  Created wheel for pytils: filename=pytils-0.4.1-py3-none-any.whl size=32868 sha256=dba01e451df2a366e2dc6388d7d704f03b4fec03eef968dafefe061e0ef5c836
  Stored in directory: /root/.cache/pip/wheels/5a/eb/7c/3b6f0c25815749883152b2caca34c35dbaab13ec2864270cbd
Successfully built tools pytils


In [5]:
# basic libraries
import os
import numpy as np
import pandas as pd
import pickle

# Transformer libraries
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss
from tqdm import tqdm_notebook, trange
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule
from sklearn.model_selection import train_test_split
from multiprocessing import Pool, cpu_count
from tools import *
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from transformers import pipeline
from transformers import  BertForMaskedLM


import logging
logging.basicConfig(level=logging.INFO)
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import random

def set_random_seeds(seed_value=100):
    # Set seed for Python's built-in random module
    random.seed(seed_value)
    
    # Set seed for NumPy
    np.random.seed(seed_value)
    
    # Set seed for PyTorch
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  # if using multiple GPUs
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Disable CuDNN benchmarking for deterministic results

# Call this function before any random operations in your code
set_random_seeds()


In [7]:
print(device)

cpu


In [8]:
def augment_text(text, aug_p=0.3):
    aug = naw.SynonymAug(aug_p=aug_p)
    augmented_text = aug.augment(text)
    return augmented_text

### Paths and Parameters

In [9]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
#DATA_DIR = "/kaggle/working/bert_inputs/"
DATA_DIR = "/kaggle/working"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-uncased'

# The name of the task to train.
TASK_NAME = 'taskforbert'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = '/kaggle/working'
#OUTPUT_DIR = '/kaggle/working/bert_outputs/'


# The directory where the evaluation reports will be written to.
REPORTS_DIR = '/kaggle/working/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = '/kaggle/working/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 2
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

output_mode = OUTPUT_MODE
cache_dir = CACHE_DIR

In [10]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)
    
print (REPORTS_DIR)

/kaggle/working//report_1


In [11]:
#DATA_DIR = "/kaggle/input/workplace-benov-dataset/workplace_benov_dataset_FINAL.csv"

#df = pd.read_csv('/kaggle/input/cgpt3bsnsdata1/C-GPT3 - Benov sx data_1sheet.csv')
df = pd.read_csv('/kaggle/input/cgpt3workplacecombined/C-GPT3 - Benov sx data_1sheet_workplace_combined.csv')
df.head(3)

,tweet_id,sentence,Label_bias
0,1001,Women are naturally more nurturing than men.,1
1,1002,Women are better suited for care-giving roles.,1
2,1003,Women are more emotional and sensitive than men.,1


In [12]:
df_bert = pd.DataFrame({'tweet_id':df['tweet_id'],
                        'label':df['Label_bias'],
                        "alpha":['a']*df.shape[0],
                        'text':df['sentence']})

df_bert.head(3)

,tweet_id,label,alpha,text
0,1001,1,a,Women are naturally more nurturing than men.
1,1002,1,a,Women are better suited for care-giving roles.
2,1003,1,a,Women are more emotional and sensitive than men.


In [13]:
!ls /usr/share/nltk_data/corpora/wordnet.zip
import zipfile
with zipfile.ZipFile("/usr/share/nltk_data/corpora/wordnet.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working/nltk_data/corpora/")

/usr/share/nltk_data/corpora/wordnet.zip


In [14]:


df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.2)
#df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.05)

#df_bert_train.head(3)
#df_bert_dev.head(3)

#df_bert_train.to_csv('/kaggle/working/bert_inputs/train.tsv', sep='\t', index=False, header=False)
#df_bert_dev.to_csv('/kaggle/working/bert_inputs/dev.tsv', sep='\t', index=False, header=False)

# Define the augmentation probability

# turn off augementation to compare
#augmentation_probability = 0

# Original augmentation probability
augmentation_probability = 0.1


# Augment the training data
augmented_texts = []
for text in df_bert_train['text']:
    augmented_text = augment_text(text, aug_p=augmentation_probability)
    augmented_texts.append(augmented_text)

# Create a new DataFrame with augmented data
augmented_df_bert_train = pd.DataFrame({
    'tweet_id': df_bert_train['tweet_id'],
    'label': df_bert_train['label'],
    'alpha': ['a'] * len(augmented_texts),
    'text': augmented_texts
})

# Concatenate original and augmented data (REAL CODE)
final_df_bert_train = pd.concat([df_bert_train, augmented_df_bert_train], ignore_index=True)

#bypass augmentation - case #2 
#final_df_bert_train = df_bert_train

# Shuffle the final training DataFrame
final_df_bert_train = final_df_bert_train.sample(frac=1).reset_index(drop=True)

# Split augmented training data into train and dev sets
augmented_train_size = int(len(final_df_bert_train) * 0.8)
final_df_bert_dev = final_df_bert_train[augmented_train_size:]
final_df_bert_train = final_df_bert_train[:augmented_train_size]

# Save augmented training and dev data
final_df_bert_train.to_csv('/kaggle/working/train.tsv', sep='\t', index=False, header=False)
final_df_bert_dev.to_csv('/kaggle/working/dev.tsv', sep='\t', index=False, header=False)



#df_bert_train.to_csv('/kaggle/working/train.tsv', sep='\t', index=False, header=False)
#df_bert_dev.to_csv('/kaggle/working/dev.tsv', sep='\t', index=False, header=False)

## Model Tuning



In [15]:
from __future__ import absolute_import, division, print_function

import csv
import os
import sys

csv.field_size_limit(2147483647) # Increase CSV reader's field limit incase we have long text.


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryClassificationProcessor(DataProcessor):
    """Processor for binary classification dataset."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            try:
                text_a = line[3]
                label = line[1]
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
            except:
                pass
        return examples


In [16]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def convert_example_to_feature(example_row):
    # return example_row
    example, label_map, max_seq_length, tokenizer, output_mode = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_ids = [0] * len(tokens)

    if tokens_b:
        tokens += tokens_b + ["[SEP]"]
        segment_ids += [1] * (len(tokens_b) + 1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding
    segment_ids += padding

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                         input_mask=input_mask,
                         segment_ids=segment_ids,
                         label_id=label_id)


In [17]:
print (DATA_DIR)
processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
#print(train_examples)
train_examples_len = len(train_examples)
print (train_examples_len)

/kaggle/working
3907


In [18]:
label_list = processor.get_labels()
num_labels = len(label_list)
print (num_labels)

num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

print (num_train_optimization_steps)

2
324


In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 423325.04B/s]


In [20]:
label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 3907 examples..
Spawning 3 processes..


  0%|          | 0/3907 [00:00<?, ?it/s]

In [21]:
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

In [22]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)

100%|██████████| 407873900/407873900 [00:32<00:00, 12398182.31B/s]


In [23]:
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [24]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)

global_step = 0
nb_tr_steps = 0
tr_loss = 0

8) Converting features to tensors

In [25]:
logger = logging.getLogger("logger")

logger.info("***** Running training *****")
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float)

9) Preparing the data-loader

In [26]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [27]:
def train_model ():
    global_step = 0
    model.train()
    for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch

            logits = model(input_ids, segment_ids, input_mask, labels=None)

            if OUTPUT_MODE == "classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
            elif OUTPUT_MODE == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), label_ids.view(-1))

            if GRADIENT_ACCUMULATION_STEPS > 1:
                loss = loss / GRADIENT_ACCUMULATION_STEPS

            loss.backward()
            print("\r%f" % loss, end='')

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

In [28]:
train_model ()

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

0.076001

Epoch:  50%|█████     | 1/2 [38:56<38:56, 2336.14s/it]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

0.005793

Epoch: 100%|██████████| 2/2 [1:17:27<00:00, 2323.79s/it]


## Model evaluation



In [29]:
def get_eval_report(task_name, labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "task": task_name,
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "accuracy": (tp+tn)/(tp+tn+fp+fn),
        "f1_score": f1_score(labels, preds),
        "precision_score": precision_score(labels, preds),        
        "recall" : recall_score(labels, preds)
    }

def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(task_name, labels, preds)

In [30]:
### Preparing evaluation set input for BERT model (Like we did earlier)
eval_examples = processor.get_dev_examples(DATA_DIR)
eval_examples_len = len(eval_examples)
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]

process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))
        
eval_all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
eval_all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
eval_all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

if OUTPUT_MODE == "classification":
    eval_all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    eval_all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.float)
    
eval_data = TensorDataset(eval_all_input_ids, eval_all_input_mask, eval_all_segment_ids, eval_all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

Preparing to convert 977 examples..
Spawning 3 processes..


  0%|          | 0/977 [00:00<?, ?it/s]

In [31]:
def model_evaluate(count, threshold) :
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    preds = []
    # Initialize a list to store wrongly predicted examples
    wrong_predictions = []

    for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask, labels=None)

        # create eval loss and other metric required by the task
        if OUTPUT_MODE == "classification":
            loss_fct = CrossEntropyLoss()
            tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
        elif OUTPUT_MODE == "regression":
            loss_fct = MSELoss()
            tmp_eval_loss = loss_fct(logits.view(-1), label_ids.view(-1))

        eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1

 

            # Convert logits to probabilities and get predicted labels
        probabilities = torch.nn.functional.softmax(logits, dim=1)
        #print(probabilities)
               
        # Apply the threshold to predicted labels
        predicted_labels = (probabilities[:, 1] >= threshold).long()  # Assuming binary classification (1 for positive class)

        #predicted_labels = torch.argmax(probabilities, dim=1)

        # Compare predicted labels with ground truth labels
        for i in range(len(label_ids)):
            if predicted_labels[i] != label_ids[i]:
                wrong_predictions.append({
                    'input_ids': input_ids[i].cpu().numpy(),
                    'predicted_label': predicted_labels[i].item(),
                    'true_label': label_ids[i].item(),
                })

        if len(preds) == 0:
            preds.append(logits.detach().cpu().numpy())
        else:
            preds[0] = np.append(
                preds[0], logits.detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    preds = preds[0]
    if OUTPUT_MODE == "classification":
        preds = np.argmax(preds, axis=1)
    elif OUTPUT_MODE == "regression":
        preds = np.squeeze(preds)
    result = compute_metrics(TASK_NAME, eval_all_label_ids.numpy(), preds)

    result['eval_loss'] = eval_loss

    output_eval_file = os.path.join(REPORTS_DIR, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")
        for key in (result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))
            print("{} ->".format(key), result[key])

    # Print information about wrong predictions
    ##print("\nWrong Predictions:")
    #for wp in wrong_predictions:
    #    print("Input IDs:", wp['input_ids'])
    #    print("Predicted Label:", wp['predicted_label'])
    #    print("True Label:", wp['true_label'])
    #    print("=" * 20)



    wrong_predictions_with_sentences = []

    for wrong_prediction in wrong_predictions:
        input_ids = wrong_prediction["input_ids"]
        input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        #input_tokens = [token for token in input_tokens if token not in ["[CLS]", "[SEP]", "[PAD]"]]
        input_tokens = [token.replace("[", "").replace("]", "").replace("'", "").replace("##", "") if token not in ["[CLS]", "[SEP]", "[PAD]"] else token for token in input_tokens]
        input_sentence = " ".join(input_tokens)  # Join tokens with spaces
        #input_sentence = tokenizer.convert_tokens_to_string(input_tokens) 
        wrong_prediction["input_sentence"] = input_sentence
        wrong_predictions_with_sentences.append(wrong_prediction)




    # Save the wrong predictions with sentences to a file
    #wrong_predictions_file = os.path.join(REPORTS_DIR, "wrong_predictions.txt")

    # Construct the filename with the count
    wrong_predictions_file = os.path.join(REPORTS_DIR, f"wrong_predictions_{count}.txt")


    #for wrong_prediction in wrong_predictions_with_sentences:
    #    print("Input Sentence: {}\n".format(wrong_prediction["input_sentence"]))
    #    print("True Label: {}\n".format(wrong_prediction["true_label"]))
    #    print("Predicted Label: {}\n".format(wrong_prediction["predicted_label"]))
    #    print("\n")


    #with open(wrong_predictions_file, "w") as writer:
    #    for wrong_prediction in wrong_predictions_with_sentences:
    #        writer.write("Input Sentence: {}\n".format(wrong_prediction["input_sentence"]))
    #        writer.write("True Label: {}\n".format(wrong_prediction["true_label"]))
    #        writer.write("Predicted Label: {}\n".format(wrong_prediction["predicted_label"]))
    #        writer.write("\n")

    # Assuming wrong_predictions_with_sentences is a list of dictionaries with keys "input_sentence", "true_label", and "predicted_label"

    # Print the header row
    print("number of wrong predictions", len(wrong_predictions_with_sentences) )
    print("Sentence\tTrue Label\tPredicted Label")
    for wrong_prediction in wrong_predictions_with_sentences:
        # Print each wrong prediction as a single row
        print("{}\t{}\t{}".format(wrong_prediction["input_sentence"], wrong_prediction["true_label"], wrong_prediction["predicted_label"]))


    # Assuming wrong_predictions_with_sentences is a list of dictionaries with keys "input_sentence", "true_label", and "predicted_label"
    with open(wrong_predictions_file, "w") as writer:
        # Write the header row
        writer.write("Sentence\tTrue Label\tPredicted Label\n")

        for wrong_prediction in wrong_predictions_with_sentences:
            # Write each wrong prediction as a single row
            writer.write("{}\t{}\t{}\n".format(wrong_prediction["input_sentence"], wrong_prediction["true_label"], wrong_prediction["predicted_label"]))



In [32]:
MAX_COUNT = 4
THRESHOLD_INIT = 0.5

threshold = THRESHOLD_INIT
model_evaluate(0,threshold)

for i in range(1, MAX_COUNT + 1):
    threshold = threshold + 0.1
    print("threshold..", threshold)
    model_evaluate(i,threshold)


Evaluating:   0%|          | 0/31 [00:00<?, ?it/s]

task -> ngs
mcc -> 0.9713033863463821
tp -> 501
tn -> 462
fp -> 8
fn -> 6
accuracy -> 0.9856704196519959
f1_score -> 0.9862204724409449
precision_score -> 0.9842829076620825
recall -> 0.9881656804733728
eval_loss -> 0.0566683882291639
number of wrong predictions 14
Sentence	True Label	Predicted Label
[CLS] don  t treat women like an object . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

Evaluating:   0%|          | 0/31 [00:00<?, ?it/s]

task -> ngs
mcc -> 0.9713033863463821
tp -> 501
tn -> 462
fp -> 8
fn -> 6
accuracy -> 0.9856704196519959
f1_score -> 0.9862204724409449
precision_score -> 0.9842829076620825
recall -> 0.9881656804733728
eval_loss -> 0.0566683882291639
number of wrong predictions 15
Sentence	True Label	Predicted Label
[CLS] don  t treat women like an object . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

Evaluating:   0%|          | 0/31 [00:00<?, ?it/s]

task -> ngs
mcc -> 0.9713033863463821
tp -> 501
tn -> 462
fp -> 8
fn -> 6
accuracy -> 0.9856704196519959
f1_score -> 0.9862204724409449
precision_score -> 0.9842829076620825
recall -> 0.9881656804733728
eval_loss -> 0.0566683882291639
number of wrong predictions 15
Sentence	True Label	Predicted Label
[CLS] don  t treat women like an object . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

Evaluating:   0%|          | 0/31 [00:00<?, ?it/s]

task -> ngs
mcc -> 0.9713033863463821
tp -> 501
tn -> 462
fp -> 8
fn -> 6
accuracy -> 0.9856704196519959
f1_score -> 0.9862204724409449
precision_score -> 0.9842829076620825
recall -> 0.9881656804733728
eval_loss -> 0.0566683882291639
number of wrong predictions 18
Sentence	True Label	Predicted Label
[CLS] don  t treat women like an object . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

Evaluating:   0%|          | 0/31 [00:00<?, ?it/s]

task -> ngs
mcc -> 0.9713033863463821
tp -> 501
tn -> 462
fp -> 8
fn -> 6
accuracy -> 0.9856704196519959
f1_score -> 0.9862204724409449
precision_score -> 0.9842829076620825
recall -> 0.9881656804733728
eval_loss -> 0.0566683882291639
number of wrong predictions 25
Sentence	True Label	Predicted Label
[CLS] don  t treat women like an object . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [33]:
#Attempting single text row 

## not functional 

from transformers import BertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from multiprocessing import cpu_count
from tqdm import tqdm

# Example text
example_text = "women should only do cleaning is wrong"
label_map = 0

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to process example text and prepare it for evaluation
def prepare_example(example_text, label_map, tokenizer, max_seq_length, output_mode):
    # Tokenize the example text
    tokens = tokenizer.tokenize(example_text)
    
    # Add [CLS] and [SEP] tokens
    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    
    # Convert tokens to input_ids and segment_ids
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    segment_ids = [0] * len(input_ids)
    
    # Pad or truncate input to max_seq_length
    padding_length = max_seq_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [0] * padding_length
        segment_ids = segment_ids + [0] * padding_length
    else:
        input_ids = input_ids[:max_seq_length]
        segment_ids = segment_ids[:max_seq_length]
    
    # Create input_mask
    input_mask = [1] * len(input_ids)
    
    # Create label_id (for classification)
    if output_mode == "classification":
        label_id = torch.tensor(label_map, dtype=torch.long)  # Set an example label here
    else:
        label_id = torch.tensor(label_map, dtype=torch.float)  # Set an example label here
    
    return input_ids, input_mask, segment_ids, label_id

# Set your desired max sequence length and output mode
MAX_SEQ_LENGTH = 128
OUTPUT_MODE = "classification"  # Change this to "regression" if needed

# Process the example text
input_ids, input_mask, segment_ids, label_id = prepare_example(
    example_text, label_map, tokenizer, MAX_SEQ_LENGTH, OUTPUT_MODE)

# Create a DataLoader for the single example
eval_data = TensorDataset(
    torch.tensor([input_ids], dtype=torch.long),
    torch.tensor([input_mask], dtype=torch.long),
    torch.tensor([segment_ids], dtype=torch.long),
    torch.tensor([label_id], dtype=torch.long if OUTPUT_MODE == "classification" else torch.float))

eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=1)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [34]:
#torch.save(model, '/kaggle/working/BERT_BS_save')
#model.save('/kaggle/working/BERT_BS_save')

# Specify the file path including the file name and extension
file_path = '/kaggle/working/BERT_BS_save.pth'

# Save the model to the specified file path
torch.save(model, file_path)








In [ ]:
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from lime.lime_text import LimeTextExplainer

# Load your BERT model and tokenizer
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Specify the file path where the model is saved
file_path = '/kaggle/working/BERT_BS_save.pth'


# Load the model
model = torch.load(file_path, map_location=torch.device('cpu'))  # Load the model on CPU


# Define a function to predict using the BERT model
def predict_fn(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        logits = model(**inputs)
    #probabilities = torch.softmax(outputs.logits, dim=1)
            # Convert logits to probabilities and get predicted labels
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    #predicted_labels = torch.argmax(probabilities, dim=1)
    return probabilities.numpy()

# Define an example text
#text = "People are so talented, but they're also very emotional"

text = "Let's not discuss Mother in Heaven because false doctrine has been taught about her"


# Create a LIME text explainer
explainer = LimeTextExplainer(class_names=["Not Benevolent", "Yes Benevolent"])

# Explain the BERT model's prediction
explanation = explainer.explain_instance(text, predict_fn, num_features=10)

# Print the explanation
explanation.show_in_notebook(text=True)
